In [47]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [57]:
df = pd.read_csv('credit_score_ds_1.csv')
df.head()

,month,age,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,num_of_loan,num_credit_inquiries,credit_history_age,amount_invested_monthly,...,occupation_Musician,occupation_Scientist,occupation_Teacher,occupation_Writer,payment_behaviour_High_spent_Large_value_payments,payment_behaviour_High_spent_Medium_value_payments,payment_behaviour_High_spent_Small_value_payments,payment_behaviour_Low_spent_Large_value_payments,payment_behaviour_Low_spent_Medium_value_payments,payment_behaviour_Low_spent_Small_value_payments
0,1,23.0,19114.12,1824.843333,3.0,4.0,4.0,4.0,265.0,21.46538,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2,23.0,19114.12,1824.843333,3.0,4.0,4.0,4.0,266.0,21.46538,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,23.0,19114.12,1824.843333,3.0,4.0,4.0,4.0,267.0,21.46538,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,23.0,19114.12,1824.843333,3.0,4.0,4.0,4.0,268.0,21.46538,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,23.0,19114.12,1824.843333,3.0,4.0,4.0,4.0,269.0,21.46538,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [99]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import numpy as np

# Определение нейронной сети
class MoreComplexClassifier(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size4, output_size):
        super(MoreComplexClassifier, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.layer3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.layer4 = nn.Linear(hidden_size3, hidden_size4)
        self.relu4 = nn.ReLU()
        self.layer5 = nn.Linear(hidden_size4, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.layer3(x)
        x = self.relu3(x)
        x = self.layer4(x)
        x = self.relu4(x)
        x = self.layer5(x)
        x = self.sigmoid(x)
        return x



In [75]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

x = df.drop(['month', 'credit_score'], axis=1)
y = np.array(df['credit_score'])

scaler = MinMaxScaler()
scaler.fit(x)
t_x = scaler.transform(x)

X_train, X_temp, y_train, y_temp = train_test_split(t_x, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [76]:
# Преобразование данных в тензоры PyTorch
X_val = torch.FloatTensor(X_val)
y_val = torch.FloatTensor(y_val)
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)

In [103]:
# Задаем параметры
input_size = 31  # Размерность входных данных
hidden_size1 = 200  # Количество нейронов в первом скрытом слое
hidden_size2 = 200  # Количество нейронов во втором скрытом слое
hidden_size3 = 100
hidden_size4 = 50
output_size = 1  # Количество выходных нейронов (бинарная классификация)

# Создаем экземпляр модели
model = MoreComplexClassifier(input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size4, output_size)

# Определяем функцию потерь и оптимизатор
criterion = nn.BCELoss()  # Бинарная кросс-энтропия для бинарной классификации
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)





In [101]:

# Обучение модели
num_epochs = 500
for epoch in range(num_epochs):
    # Прямой проход
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))

    # Обратный проход и оптимизация
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Выводим значение функции потерь на каждой эпохе
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/500], Loss: 0.5775
Epoch [20/500], Loss: 0.5183
Epoch [30/500], Loss: 0.4966
Epoch [40/500], Loss: 0.4878
Epoch [50/500], Loss: 0.4818
Epoch [60/500], Loss: 0.4783
Epoch [70/500], Loss: 0.4773
Epoch [80/500], Loss: 0.4749
Epoch [90/500], Loss: 0.4730
Epoch [100/500], Loss: 0.4749
Epoch [110/500], Loss: 0.4693
Epoch [120/500], Loss: 0.4668
Epoch [130/500], Loss: 0.4753
Epoch [140/500], Loss: 0.4686
Epoch [150/500], Loss: 0.4639
Epoch [160/500], Loss: 0.4651
Epoch [170/500], Loss: 0.4626
Epoch [180/500], Loss: 0.4640
Epoch [190/500], Loss: 0.4631
Epoch [200/500], Loss: 0.4713
Epoch [210/500], Loss: 0.4627
Epoch [220/500], Loss: 0.4617
Epoch [230/500], Loss: 0.4598
Epoch [240/500], Loss: 0.4585
Epoch [250/500], Loss: 0.4641
Epoch [260/500], Loss: 0.4583
Epoch [270/500], Loss: 0.4631
Epoch [280/500], Loss: 0.4592
Epoch [290/500], Loss: 0.4551
Epoch [300/500], Loss: 0.4625
Epoch [310/500], Loss: 0.4556
Epoch [320/500], Loss: 0.4524
Epoch [330/500], Loss: 0.4620
Epoch [340/500], Lo

In [102]:
# Оценка точности с использованием AUC на тестовом наборе
model.eval()
with torch.no_grad():
    y_pred_probs_test = model(X_test)
    y_pred_probs_train = model(X_train)
    y_pred_probs_test = y_pred_probs_test.numpy().flatten()
    y_pred_probs_train = y_pred_probs_train.numpy().flatten()
    auc_score_test = roc_auc_score(y_test.numpy(), y_pred_probs_test)
    auc_score_train = roc_auc_score(y_train.numpy(), y_pred_probs_train)
    
print(f'AUC на тренировочном наборе: {auc_score_train:.4f}')
print(f'AUC на тестовом наборе: {auc_score_test:.4f}')


AUC на тренировочном наборе: 0.8253
AUC на тестовом наборе: 0.8175


In [65]:
input_size = 31  # Размерность входных данных
hidden_size1 = 100  # Количество нейронов в первом скрытом слое
hidden_size2 = 100  # Количество нейронов во втором скрытом слое
hidden_size3 = 50
output_size = 1  # Количество выходных нейронов (бинарная классификация)


AUC на тренировочном наборе: 0.8251
AUC на тестовом наборе: 0.8169

In [106]:
hidden_layer_sizes1 = [i for i in np.arange(5, 301, 10)]
hidden_layer_sizes2 = [i for i in np.arange(5, 301, 10)]
hidden_layer_sizes3 = [i for i in np.arange(5, 301, 10)]
hidden_layer_sizes4 = [i for i in np.arange(5, 301, 10)]
num_epochs_list = [i for i in np.arange(50, 1000, 50)]


In [107]:
best_auc = 0.0
best_hidden_size = None
best_num_epochs = None
for hidden_size1 in hidden_layer_sizes1:
    for hidden_size2 in hidden_layer_sizes2:
        for hidden_size3 in hidden_layer_sizes3:
            for hidden_size4 in hidden_layer_sizes4:
                for num_epochs in num_epochs_list:
                    outputs = model(X_train)
                    loss = criterion(outputs, y_train.view(-1, 1))
                
                    # Обратный проход и оптимизация
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                
                    # Выводим значение функции потерь на каждой эпохе
                    if (epoch + 1) % 10 == 0:
                        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
                # Оценка точности с использованием AUC на тестовом наборе
                model.eval()
                with torch.no_grad():
                    y_pred_probs_test = model(X_test)
                    y_pred_probs_train = model(X_train)
                    y_pred_probs_test = y_pred_probs_test.numpy().flatten()
                    y_pred_probs_train = y_pred_probs_train.numpy().flatten()
                    auc_score_test = roc_auc_score(y_test.numpy(), y_pred_probs_test)
                    auc_score_train = roc_auc_score(y_train.numpy(), y_pred_probs_train)

                # Сравнение с текущим лучшим результатом
                if auc_score_test > best_auc:
                    best_auc = auc_score_test
                    best_hidden_sizes = (hidden_size1, hidden_size2, hidden_size3, hidden_size4)
                    best_num_epochs = num_epochs

print(f'Лучший AUC: {best_auc:.4f}')
print(f'Лучшие размеры скрытых слоев: {best_hidden_sizes}')
print(f'Лучшее количество эпох: {best_num_epochs}')

Epoch [500/50], Loss: 0.6770
Epoch [500/100], Loss: 0.6359
Epoch [500/150], Loss: 0.5973
Epoch [500/200], Loss: 0.6240
Epoch [500/250], Loss: 0.5820
Epoch [500/300], Loss: 0.5901
Epoch [500/350], Loss: 0.5881
Epoch [500/400], Loss: 0.5713
Epoch [500/450], Loss: 0.5492
Epoch [500/500], Loss: 0.5502
Epoch [500/550], Loss: 0.5364
Epoch [500/600], Loss: 0.5242
Epoch [500/650], Loss: 0.5294
Epoch [500/700], Loss: 0.5242
Epoch [500/750], Loss: 0.5165
Epoch [500/800], Loss: 0.5218
Epoch [500/850], Loss: 0.5164
Epoch [500/900], Loss: 0.5117
Epoch [500/950], Loss: 0.5138
Epoch [500/50], Loss: 0.5093
Epoch [500/100], Loss: 0.5054
Epoch [500/150], Loss: 0.5072
Epoch [500/200], Loss: 0.5024
Epoch [500/250], Loss: 0.5033
Epoch [500/300], Loss: 0.4992
Epoch [500/350], Loss: 0.4963
Epoch [500/400], Loss: 0.4955
Epoch [500/450], Loss: 0.4936
Epoch [500/500], Loss: 0.4950
Epoch [500/550], Loss: 0.4930
Epoch [500/600], Loss: 0.4933
Epoch [500/650], Loss: 0.4915
Epoch [500/700], Loss: 0.4918
Epoch [500/7

KeyboardInterrupt: 

In [108]:
print(f'Лучший AUC: {best_auc:.4f}')
print(f'Лучшие размеры скрытых слоев: {best_hidden_sizes}')
print(f'Лучшее количество эпох: {best_num_epochs}')

Лучший AUC: 0.8372
Лучшие размеры скрытых слоев: (5, 5, 35, 125)
Лучшее количество эпох: 950
